In [3]:
from typing import List, TypedDict
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings, ChatOllama  # Updated imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END

# 1) Load Documents
docs = (
    PyPDFLoader("./documents/book1.pdf").load() +
    PyPDFLoader("./documents/book2.pdf").load() +
    PyPDFLoader("./documents/book3.pdf").load()
)

# 2) Chunk & Clean
chunks = RecursiveCharacterTextSplitter(chunk_size=900, chunk_overlap=150).split_documents(docs)
for d in chunks:
    d.page_content = d.page_content.encode("utf-8", "ignore").decode("utf-8", "ignore")

# 3) Local Indexing with mxbai-embed-large
embeddings = OllamaEmbeddings(model="mxbai-embed-large")
vector_store = FAISS.from_documents(chunks, embeddings)
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 4})

# 4) Local LLM with Qwen 2.5
llm = ChatOllama(model="qwen2.5:7b-instruct", temperature=0)

# --- LangGraph Setup (Nodes remain largely the same) ---

class State(TypedDict):
    question: str
    docs: List[Document]
    answer: str

def retrieve(state):
    return {"docs": retriever.invoke(state["question"])}

def generate(state):
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Answer only from the context. If not in context, say you don't know."),
        ("human", "Question: {question}\n\nContext:\n{context}"),
    ])
    context = "\n\n".join(d.page_content for d in state["docs"])
    chain = prompt | llm
    out = chain.invoke({"question": state["question"], "context": context})
    return {"answer": out.content}

# Compile Graph
workflow = StateGraph(State)
workflow.add_node("retrieve", retrieve)
workflow.add_node("generate", generate)
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

app = workflow.compile()

# 5) Run
res = app.invoke({"question": "What is a transformer in deep learning.", "docs": [], "answer": ""})
print(res["answer"])

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).